In [1]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore')

/Users/robertoorazi/Repositories/Facultad/tuia/cuarto-cuatrimestre/aa1/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
class NeuralNetwork:
    def __init__(self, epochs=50, batch_size=16, learning_rate=0.01):
        #inicializo algunos parámetros como épocas, batch_size, learning rate
        #(no son necesarios)
        #se puede agregar la cantidad de capas, la cantidad de neuronas por capa (pensando en hacer una clase que pueda ser usada para cualquier caso)
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = None

    def build_model(self, input_shape, num_classes):
        # ejemplo con 2 capas ocultas de 64 neuronas y activación softmax (multiclase, recibe la cantidad de clases como input, además del input_shape)
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

        #compilo el modelo con el optimizador Adam, la función de pérdida categorical_crossentropy y la métrica accuracy
        #totalmente optimizable e incluso pueden ser parámetros de la función build_model
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model = model

    def train(self, X_train, y_train, X_valid, y_valid):
        # simplemente el fit del modelo. Devuelvo la evolución de la función de pérdida, ya que es interesante ver como varía a medida que aumentan las épocas!
        history=self.model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=self.epochs, batch_size=self.batch_size)
        return history.history['loss']

    def evaluate(self, X_test, y_test):
        ### evalúo en test
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f"test accuracy: {accuracy:.4f}")

    def predict(self, X_new):
        ### predicciones
        predictions = self.model.predict(X_new)
        return predictions
    

    ### acá también me podría armar una función para graficar la evolución de la función de pérdida en train y validación, etc etc

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
nn = NeuralNetwork(epochs=50, batch_size=16, learning_rate=0.01)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### en este caso tenemos más de 2 clases, por lo que necesitamos hacer one hot encoding en la salida, para este modelo en particular.
y_train=tf.keras.utils.to_categorical(y_train)
y_valid=tf.keras.utils.to_categorical(y_valid)
y_test=tf.keras.utils.to_categorical(y_test)

#buildeo el modelo
nn.build_model(input_shape=X_train.shape[1], num_classes=y_train.shape[1])

# entreno el modelo
history=nn.train(X_train, y_train, X_valid, y_valid)

# evaluo metricas
nn.evaluate(X_test, y_test)

## predicciones...

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history)
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend('train', 'validation', )
plt.show()

# regresion

In [ ]:
class NeuralNetworkRegressor:
    def __init__(self, epochs=50, batch_size=16, learning_rate=0.01):
        #inicializo algunos parámetros como épocas, batch_size, learning rate
        #(no son necesarios)
        #se puede agregar la cantidad de capas, la cantidad de neuronas por capa
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = None

    def build_model(self, input_shape):
        # ejemplo con 2 capas ocultas de 64 neuronas, la saida en este caso es solo una neurona, con activación lineal
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
            tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,), kernel_regularizer=tf.keras.regularizers.l2(0.01)),# El valor es el alpha
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(self.dropout_rate),
            tf.keras.layers.Dense(1)
        ])

        #compilo el modelo con el optimizador Adam, la función de pérdida mse y métrica mse también
        #totalmente optimizable e incluso pueden ser parámetros de la función build_model

        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

        self.model = model

    def train(self, X_train, y_train, X_valid, y_valid):
        # simplemente el fit del modelo. Devuelvo la evolución de la función de pérdida, ya que es interesante ver como varía a medida que aumentan las épocas!
        history=self.model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=self.epochs, batch_size=self.batch_size)
        return history.history['loss'], history.history['val_loss']

    def evaluate(self, X_test, y_test):
        ### evalúo en test
        loss, mse = self.model.evaluate(X_test, y_test)
        print(f"test MSE: {mse:.4f}")

    def predict(self, X_new):
        ### predicciones
        predictions = self.model.predict(X_new)
        return predictions
    

    ### acá también me podría armar una función para graficar la evolución de la función de pérdida en train y validación, etc etc

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes()
X, y = data.data, data.target
y = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train = x_scaler.fit_transform(X_train)
y_train = y_scaler.fit_transform(y_train)
X_valid = x_scaler.transform(X_valid)
y_valid = y_scaler.transform(y_valid)
X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test)

nnr= NeuralNetworkRegressor(epochs=50, batch_size=16, learning_rate=0.01)
nnr.build_model(input_shape=X_train.shape[1], num_classes=y_train.shape[1])
history=nnr.train(X_train, y_train, X_valid, y_valid)

# evaluo metricas
nnr.evaluate(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history)

plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
